<a href="https://colab.research.google.com/github/amay1212/Grokking-DL/blob/main/MNIST_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys, numpy as np
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()


11490434/11490434 [==============================] - 0s 0us/step


In [3]:

images, labels = (x_train.reshape(60000,28*28) / 255, y_train[0:60000])

one_hot_labels = np.zeros((len(labels),10))
for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels

test_images = x_test.reshape(len(x_test),28*28) / 255
test_labels = np.zeros((len(y_test),10))
for i,l in enumerate(y_test):
    test_labels[i][l] = 1

np.random.seed(1)
relu = lambda x:(x>=0) * x # returns x if x > 0, return 0 otherwise
relu2deriv = lambda x: x>=0 # returns 1 for input > 0, return 0 otherwise
alpha, iterations, hidden_size, pixels_per_image, num_labels = (0.0005, 350, 40, 784, 10)

weights_0_1 = 0.2*np.random.random((pixels_per_image, 40)) - 0.1
weights_1_2 = 0.2*np.random.random((40, 16)) - 0.1
weights_2_3 = 0.2*np.random.random((16, 10)) - 0.1
iterations = 200
for j in range(iterations):
    error, correct_cnt = (0.0, 0)

    for i in range(len(images)):
        input_layer = images[i: i+1] # (1, 784)
        target = labels[i:i+1]

        layer_1_output = relu(input_layer.dot(weights_0_1)) # (1, 784) dot (784, 2)
        #print("layer 1 o/p shape", layer_1.shape) # (1, 2)

        layer_2_output = relu(layer_1_output.dot(weights_1_2)) # (1, 2) dot (2, 4)
        #print("layer 2 o/p shape", layer_2_output.shape) # (1, 4)

        layer_3_output = layer_2_output.dot(weights_2_3) # (1, 4) dot (4, 10)
        #print("layer 3 o/p shape", layer_3_output.shape)

        # calculate error and gradient of error..
        #error += np.sum((layer_3_output - target)**2)
        # error += np.sum((target - layer_3_output)**2)
        error += np.sum((labels[i:i+1] - layer_3_output) ** 2)
        correct_cnt += int(np.argmax(layer_3_output) == \
                           np.argmax(target))
        layer_3_gradient = (layer_3_output - target)
        #layer_3_gradient = (target - layer_3_output)

        #print("grad of layer3 output aka prediction", layer_3_gradient.shape)

        #back propagation..

        layer_2_delta = layer_3_gradient.dot(weights_2_3.T) * relu2deriv(layer_2_output)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1_output)

        weights_2_3 -= alpha * (layer_2_output.T.dot(layer_3_gradient))
        weights_1_2 -= alpha* (layer_1_output.T.dot(layer_2_delta))
        weights_0_1 -= alpha * (input_layer.T.dot(layer_1_delta))
    #print('done!!')
    sys.stdout.write("\r I:"+str(j)+ \
                     " Train-Err:" + str(error/float(len(images)))[0:5] +\
                     " Train-Acc:" + str(correct_cnt/float(len(images))))

    if(j % 10 == 0 or j == iterations-1):
        error, correct_cnt = (0.0, 0)

        for i in range(len(test_images)):

            layer_0 = test_images[i:i+1]
            layer_1 = relu(np.dot(layer_0,weights_0_1))
            layer_2 = np.dot(layer_1,weights_1_2)

            layer_3 = np.dot(layer_2, weights_2_3)

            # error += np.sum((layer_3 - test_labels[i:i+1]) ** 2)
            # error += np.sum((test_labels[i:i+1] - layer_3) ** 2)
            error += np.sum((test_labels[i:i+1] - layer_3) ** 2)
            correct_cnt += int(np.argmax(layer_3) == \
                                            np.argmax(test_labels[i:i+1]))
        sys.stdout.write(" Test-Err:" + str(error/float(len(test_images)))[0:5] +\
                         " Test-Acc:" + str(correct_cnt/float(len(test_images))))
        print()



 I:0 Train-Err:0.680 Train-Acc:0.5241 Test-Err:0.863 Test-Acc:0.745
 I:10 Train-Err:0.284 Train-Acc:0.8904666666666666 Test-Err:0.746 Test-Acc:0.8619
 I:20 Train-Err:0.269 Train-Acc:0.8950333333333333 Test-Err:0.820 Test-Acc:0.8645
 I:30 Train-Err:0.264 Train-Acc:0.9002833333333333 Test-Err:0.727 Test-Acc:0.8623
 I:40 Train-Err:0.258 Train-Acc:0.90185 Test-Err:0.715 Test-Acc:0.8505
 I:50 Train-Err:0.252 Train-Acc:0.9013166666666667 Test-Err:0.802 Test-Acc:0.8527
 I:60 Train-Err:0.249 Train-Acc:0.90135 Test-Err:1.026 Test-Acc:0.8589
 I:70 Train-Err:0.247 Train-Acc:0.9007 Test-Err:1.180 Test-Acc:0.85
 I:80 Train-Err:0.250 Train-Acc:0.8983333333333333 Test-Err:1.277 Test-Acc:0.8389
 I:90 Train-Err:0.257 Train-Acc:0.8967666666666667 Test-Err:1.205 Test-Acc:0.8281
 I:100 Train-Err:0.255 Train-Acc:0.8990166666666667 Test-Err:1.131 Test-Acc:0.8497
 I:110 Train-Err:0.253 Train-Acc:0.9000666666666667 Test-Err:1.062 Test-Acc:0.8656
 I:120 Train-Err:0.252 Train-Acc:0.9017833333333334 Test-Err:1.0